<img src="https://github.com/rjpost20/Anomalous-Bank-Transactions-Detection-Project/blob/main/data/AdobeStock_319163865.jpeg?raw=true">
Image by <a href="https://stock.adobe.com/contributor/200768506/andsus?load_type=author&prev_url=detail" >AndSus</a> on Adobe Stock

# Phase 5 Project: *Detecting Anomalous Financial Transactions*

## Notebook 2: Preprocessing and Feature Engineering

### By Ryan Posternak

Flatiron School, Full-Time Live NYC<br>
Project Presentation Date: August 25th, 2022<br>
Instructor: Joseph Mata

## Goal: Build a model for FinCEN that can accurately identify anomalous financial transactions, as measured by area under the precision-recall curve (AUPRC)

*This is a project for learning purposes. FinCEN is not involved with this project in any way.*

<br>

# Imports and Reading in Data

### Google colab compatibility downloads

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz 
!tar xf spark-3.3.0-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
!pip install pyspark==3.3.0
!pip install -q findspark
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Connect to Google drive
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


### Import libraries, packages and modules

In [3]:
import numpy as np
import pandas as pd
from itertools import chain

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DoubleType, TimestampType

import seaborn as sns
from IPython.display import HTML, display
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [4]:
# Import helper functions
helper_functions = files.upload()
from helper_functions import create_weight_col, spark_resample

Saving helper_functions.py to helper_functions.py


In [5]:
# Check Colab GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Not connected to a GPU


In [6]:
# Check Colab RAM info
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [7]:
# Set text to wrap in Google colab notebook
def set_css():
    display(HTML("""
    <style>
      pre {
          white-space: pre-wrap;
      }
    </style>
    """))
get_ipython().events.register('pre_run_cell', set_css)

In [8]:
# Initialize Spark Session
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab")\
        .config("spark.ui.port", "4050")\
        .config("spark.driver.memory", "15g")\
        .getOrCreate()

spark

### Description of Features

**Dataset 1 – Transactions:**

`MessageId` - Globally unique identifier within this dataset for individual transactions<br>
`UETR` - The Unique End-to-end Transaction Reference—a 36-character string enabling traceability of all individual transactions associated with a single end-to-end transaction<br>
`TransactionReference` - Unique identifier for an individual transaction<br>
`Timestamp` - Time at which the individual transaction was initiated<br>
`Sender` - Institution (bank) initiating/sending the individual transaction<br>
`Receiver` - Institution (bank) receiving the individual transaction<br>
`OrderingAccount` - Account identifier for the originating ordering entity (individual or organization) for end-to-end transaction<br>
`OrderingName` - Name for the originating ordering entity<br>
`OrderingStreet` - Street address for the originating ordering entity<br>
`OrderingCountryCityZip` - Remaining address details for the originating ordering entity<br>
`BeneficiaryAccount` - Account identifier for the final beneficiary entity (individual or organization) for end-to-end transaction<br>
`BeneficiaryName` - Name for the final beneficiary entity<br>
`BeneficiaryStreet` - Street address for the final beneficiary entity<br>
`BeneficiaryCountryCityZip` - Remaining address details for the final beneficiary entity<br>
`SettlementDate` - Date the individual transaction was settled<br>
`SettlementCurrency` - Currency used for transaction<br>
`SettlementAmount` - Value of the transaction net of fees/transfer charges/forex<br>
`InstructedCurrency` - Currency of the individual transaction as instructed to be paid by the Sender<br>
`InstructedAmount` - Value of the individual transaction as instructed to be paid by the Sender<br>
`Label` - Boolean indicator of whether the transaction is anomalous or not. This is the target variable for the prediction task.<br>
<br>
**Dataset 2 – Banks:**

`Bank` - Identifier for the bank<br>
`Account` - Identifier for the account<br>
`Name` - Name of the account<br>
`Street` - Street address associated with the account<br>
`CountryCityZip` - Remaining address details associated with the account<br>
`Flags` - Enumerated data type indicating potential issues or special features that have been associated with an account. Flag definitions are below:<br>
00 - No flags<br>
01 - Account closed<br>
03 - Account recently opened<br>
04 - Name mismatch<br>
05 - Account under monitoring<br>
06 - Account suspended<br>
07 - Account frozen<br>
08 - Non-transaction account<br>
09 - Beneficiary deceased<br>
10 - Invalid company ID<br>
11 - Invalid individual ID<br>
<br>
Additional information from data providers:<br>
"Because each end-to-end transaction is defined by one originating orderer and one final beneficiary, the `OrderingAccount` and `BeneficiaryAccount` in a given row may not necessarily belong to the bank in that row's `Sender` and the bank in that row's `Receiver`, respectively. The correct way to associate an `OrderingAccount` to the correct bank is to identify the `Sender` bank in the originating (first) individual transaction in that end-to-end transaction, and the correct way to associate a `BeneficiaryAccount` to the correct bank is to identify the `Receiver` bank in the final (last) individual transaction in that end-to-end transaction."

## Read in Data

In [11]:
# Read in transactions training and testing data csv files to Spark DataFrames
train_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/transaction_train_dataset.csv', header=True, inferSchema=True)
test_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/transaction_test_dataset.csv', header=True, inferSchema=True)

# Read in banks data csv file to a Spark DataFrame
banks_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/bank_dataset.csv', header=True, inferSchema=True)

<br>

# Preprocessing & Feature Engineering

Steps:
1. Create `OrderingCountry` and `BeneficiaryCountry` features to be used for `OrderingCountryFreq` and `BeneficiaryCountryFreq`
2. Create `InstructedAmountUSD` feature by converting all currencies in `InstructedAmount` to USD-scale
3. Create `IntermediaryTransactions` feature by counting number of repeated UETRs for each end-to-end transaction
4. Create `OriginalSender` and `FinalReceiver` features
5. Create `Flagged` feature by joining `Flags` column of `banks_df`
6. Create `OrderingCountryFreq` and `BeneficiaryCountryFreq` features
7. Create `SenderHourFreq` feature
8. Create `SenderCurrencyFreq` and `SenderCurrencyAmtAvg` features
9. Create `SenderFreq` and `ReceiverFreq` features
9. Create `SenderReceiverFreq` feature

## Create `OrderingCountry` and `BeneficiaryCountry` features

In [12]:
# Split the content of OrderingCountryCityZip and BeneficiaryCountryCityZip on the forward-slash character ('/')
split_cols_ordering_train = F.split(train_df['OrderingCountryCityZip'], '/')
split_cols_beneficiary_train = F.split(train_df['BeneficiaryCountryCityZip'], '/')

# Add the OrderingCountry as a column to train_df and test_df
train_df = train_df.withColumn('OrderingCountry', split_cols_ordering_train.getItem(0))
train_df = train_df.withColumn('BeneficiaryCountry', split_cols_beneficiary_train.getItem(0))

# Repeat for test_df
split_cols_ordering_test = F.split(test_df['OrderingCountryCityZip'], '/')
split_cols_beneficiary_test = F.split(test_df['BeneficiaryCountryCityZip'], '/')
test_df = test_df.withColumn('OrderingCountry', split_cols_ordering_test.getItem(0))
test_df = test_df.withColumn('BeneficiaryCountry', split_cols_beneficiary_test.getItem(0))

### Accuracy check

In [ ]:
# Retrieve three random UETR codes
sample = train_df.sample(False, 0.00001).limit(3).collect()
UETRs = [sample[row]['UETR'] for row in range(3)]

# Only show relevant columns
cols_to_show = ['UETR', 'OrderingCountryCityZip', 'OrderingCountry', 'BeneficiaryCountryCityZip', 'BeneficiaryCountry']

# Preview transactions
train_df.filter(train_df.UETR.isin(UETRs)).select(cols_to_show).show(truncate=False)

In [ ]:
# Display value counts for new 'OrderingCountry' and 'BeneficiaryCountry' features of train_df
ordering_country_counts = train_df.groupBy('OrderingCountry').count().withColumn('percent', F.col('count')/train_df.count())
beneficiary_country_counts = train_df.groupBy('BeneficiaryCountry').count().withColumn('percent', F.col('count')/train_df.count())

ordering_country_counts.show(28, truncate=False)
beneficiary_country_counts.show(truncate=False)

**Remarks:**
- It looks like there are some rows with incorrect/unusual values, but for most of them it looks pretty obvious what the intended value was supposed to be. A couple are a bit ambiguous, e.g. 'E' in `OrderingCountries` could be either `DE`, `ES`, `EC` or `PE`. For these ones, we'll just impute the most frequent likely intended value.

In [13]:
# Replace incorrect/unusual ordering country values with most likely intended value
train_df = train_df.replace('F', 'FR', subset='OrderingCountry')
train_df = train_df.replace('GBS1 1TD', 'GB', subset='OrderingCountry')
train_df = train_df.replace('B', 'GB', subset='OrderingCountry')
train_df = train_df.replace('GBE5C 7TG', 'GB', subset='OrderingCountry')
train_df = train_df.replace('U', 'US', subset='OrderingCountry')
train_df = train_df.replace('D', 'DE', subset='OrderingCountry')
train_df = train_df.replace('R', 'FR', subset='OrderingCountry')
train_df = train_df.replace('GBBD6 2YA', 'GB', subset='OrderingCountry')
train_df = train_df.replace('E', 'ES', subset='OrderingCountry')
train_df = train_df.replace('FR46216 JOSEPH', 'FR', subset='OrderingCountry')
train_df = train_df.replace('FR97705 PEREIRA', 'FR', subset='OrderingCountry')
train_df = train_df.replace('G', 'GB', subset='OrderingCountry')
train_df = train_df.replace('S', 'US', subset='OrderingCountry')
train_df = train_df.replace('DE51111 WITTSTOCK', 'DE', subset='OrderingCountry')

# Replace incorrect/unusual beneficiary country values with most likely intended value
train_df = train_df.replace('E', 'ES', subset='BeneficiaryCountry')
train_df = train_df.replace('D', 'DE', subset='BeneficiaryCountry')
train_df = train_df.replace('USMACKENZIESTAD| KS 08333', 'US', subset='BeneficiaryCountry')
train_df = train_df.replace('USMICHAELVIEW| NH 97912', 'US', subset='BeneficiaryCountry')
train_df = train_df.replace('U', 'US', subset='BeneficiaryCountry')
train_df = train_df.replace('G', 'GB', subset='BeneficiaryCountry')
train_df = train_df.replace('USEMILYVIEW| KS 16890', 'US', subset='BeneficiaryCountry')
train_df = train_df.replace('USAGUILARSIDE| KY 08409', 'US', subset='BeneficiaryCountry')

# Display new value counts for 'OrderingCountry' and 'BeneficiaryCountry' features of train_df
ordering_country_counts = train_df.groupBy('OrderingCountry').count().withColumn('percent', F.col('count')/train_df.count())
beneficiary_country_counts = train_df.groupBy('BeneficiaryCountry').count().withColumn('percent', F.col('count')/train_df.count())

ordering_country_counts.show(truncate=False)
beneficiary_country_counts.show(truncate=False)

+---------------+-------+--------------------+
|OrderingCountry|count  |percent             |
+---------------+-------+--------------------+
|PT             |36690  |0.007820151436838263|
|GB             |529859 |0.11293479477164582 |
|DE             |603157 |0.12855762006511465 |
|ES             |80991  |0.017262520714662517|
|EC             |9547   |0.002034859246865492|
|TR             |2699   |5.752681582999856E-4|
|ZA             |514627 |0.10968822767745338 |
|US             |1002612|0.21369794691717864 |
|FR             |1621628|0.3456357736227081  |
|CH             |25027  |0.005334285364125135|
|IT             |49092  |0.010463528872642791|
|PE             |73733  |0.01571554172505848 |
|JP             |75057  |0.015997740703046322|
|ZW             |67006  |0.014281740724360443|
+---------------+-------+--------------------+

+------------------+-------+--------------------+
|BeneficiaryCountry|count  |percent             |
+------------------+-------+--------------------+
|ID

### Repeat for `test_df`

In [14]:
# Display value counts for new 'OrderingCountry' and 'BeneficiaryCountry' features of test_df
ordering_country_counts = test_df.groupBy('OrderingCountry').count().withColumn('percent', F.col('count')/test_df.count())
beneficiary_country_counts = test_df.groupBy('BeneficiaryCountry').count().withColumn('percent', F.col('count')/test_df.count())

ordering_country_counts.show(28, truncate=False)
beneficiary_country_counts.show(truncate=False)

+---------------+------+---------------------+
|OrderingCountry|count |percent              |
+---------------+------+---------------------+
|F              |1     |1.4182224567016683E-6|
|PT             |5861  |0.008312201818728479 |
|GB             |79044 |0.11210197586752668  |
|DE             |90321 |0.1280952705117514   |
|ES             |11998 |0.017015833035506616 |
|EC             |1361  |0.0019302007635709707|
|TR             |413   |5.85725874617789E-4  |
|ZA             |76718 |0.10880319043323859  |
|US             |150910|0.21402395094084878  |
|FR             |245023|0.3474971210084129   |
|CH             |3798  |0.005386408890552937 |
|IT             |7369  |0.010450881283434594 |
|PE             |11095 |0.015735178157105012 |
|JP             |11150 |0.015813180392223604 |
|ZW             |10042 |0.014241789910198154 |
|R              |2     |2.8364449134033366E-6|
|D              |1     |1.4182224567016683E-6|
|S              |1     |1.4182224567016683E-6|
+------------

In [15]:
# Replace incorrect/unusual ordering country values with most likely intended value
test_df = test_df.replace('F', 'FR', subset='OrderingCountry')
test_df = test_df.replace('R', 'FR', subset='OrderingCountry')
test_df = test_df.replace('D', 'DE', subset='OrderingCountry')
test_df = test_df.replace('S', 'US', subset='OrderingCountry')

# Replace incorrect/unusual beneficiary country values with most likely intended value
test_df = test_df.replace('U', 'US', subset='BeneficiaryCountry')
test_df = test_df.replace('USDPO AA 17341', 'US', subset='BeneficiaryCountry')
test_df = test_df.replace('USFPO AE 29185', 'US', subset='BeneficiaryCountry')
test_df = test_df.replace('S', 'US', subset='BeneficiaryCountry')

# Display new value counts for 'OrderingCountry' and 'BeneficiaryCountry' features of test_df
ordering_country_counts = test_df.groupBy('OrderingCountry').count().withColumn('percent', F.col('count')/test_df.count())
beneficiary_country_counts = test_df.groupBy('BeneficiaryCountry').count().withColumn('percent', F.col('count')/test_df.count())

ordering_country_counts.show(truncate=False)
beneficiary_country_counts.show(truncate=False)

+---------------+------+---------------------+
|OrderingCountry|count |percent              |
+---------------+------+---------------------+
|PT             |5861  |0.008312201818728479 |
|GB             |79044 |0.11210197586752668  |
|DE             |90322 |0.1280966887342081   |
|ES             |11998 |0.017015833035506616 |
|EC             |1361  |0.0019302007635709707|
|TR             |413   |5.85725874617789E-4  |
|ZA             |76718 |0.10880319043323859  |
|US             |150911|0.21402536916330547  |
|FR             |245026|0.347501375675783    |
|CH             |3798  |0.005386408890552937 |
|IT             |7369  |0.010450881283434594 |
|PE             |11095 |0.015735178157105012 |
|JP             |11150 |0.015813180392223604 |
|ZW             |10042 |0.014241789910198154 |
+---------------+------+---------------------+

+------------------+------+--------------------+
|BeneficiaryCountry|count |percent             |
+------------------+------+--------------------+
|ID   

### Visualize target class distributions or ordering countries and beneficiary countries of ordering and beneficiary entities

In [ ]:
# Sample 5% of train_df for visualizations (approximately 235k observations) for efficiency
viz_cols = ['OrderingCountry', 'BeneficiaryCountry', 'Label']
viz_df = train_df.select(viz_cols).sample(withReplacement=False, fraction=0.05, seed=42).toPandas()

In [ ]:
# Define figure and axes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(22, 9))

# Set consistent color palette where y-axis values in both axes, otherwise silver
palette = sns.color_palette('muted', as_cmap=True)*2
palette_map = {}
for val, color in zip(viz_df['OrderingCountry'].value_counts().index, palette):
    if val in viz_df[viz_df.Label == 0]['OrderingCountry'].value_counts().index \
    and val in viz_df[viz_df.Label == 1]['OrderingCountry'].value_counts().index:
        palette_map[val] = color
    else:
        palette_map[val]= 'silver'

# Plot countplot of non-anomalous transactions
sns.countplot(x='OrderingCountry', data=viz_df[viz_df.Label == 0], ax=ax1, 
              order=viz_df[viz_df.Label == 0]['OrderingCountry'].value_counts().index,  # Order descending
              palette=palette_map)

# Plot countplot of anomalous transactions
sns.countplot(x='OrderingCountry', data=viz_df[viz_df.Label == 1], ax=ax2, 
              order=viz_df[viz_df.Label == 1]['OrderingCountry'].value_counts().index,  # Order descending
              palette=palette_map)

# Print percentages on top of bars (ax1)
for p in ax1.patches:
    txt = str(round(p.get_height() / viz_df[viz_df.Label == 0].shape[0]*100, 1)) + '%'
    txt_x = p.get_x()+0.05
    txt_y = p.get_height()+350
    ax1.text(txt_x, txt_y, txt, fontsize=12)

# Print percentages on top of bars (ax2)
for p in ax2.patches:
    txt = str(round(p.get_height() / viz_df[viz_df.Label == 1].shape[0]*100, 1)) + '%'
    txt_x = p.get_x()+0.29
    txt_y = p.get_height()+0.7
    ax2.text(txt_x, txt_y, txt, fontsize=12)

ax1.set_title('Ordering Entity Countries of Non-Anomalous Transactions (Label 0)', fontsize=17)
ax1.set_xlabel('Ordering Entity Country', fontsize=16)
ax1.set_xticklabels(ax1.get_xticklabels(), fontsize=12)
ax1.set_ylabel('Count (5% Sample)', fontsize=16)
ax1.set_yticklabels(['{:,}'.format(int(x)) for x in ax1.get_yticks()], fontsize=12)
ax2.set_title('Ordering Entity Countries of Anomalous Transactions (Label 1)', fontsize=17)
ax2.set_xlabel('Ordering Entity Country', fontsize=16)
ax2.set_xticklabels(ax2.get_xticklabels(), fontsize=12)
ax2.set_ylabel('Count (5% Sample)', fontsize=16)
ax2.set_yticklabels(ax2.get_yticks().astype(int), fontsize=12);

In [ ]:
# Define figure and axes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(22, 9))

# Set consistent color palette where y-axis values in both axes, otherwise silver
palette = sns.color_palette('muted', as_cmap=True)*2
palette_map = {}
for val, color in zip(viz_df['BeneficiaryCountry'].value_counts().index, palette):
    if val in viz_df[viz_df.Label == 0]['BeneficiaryCountry'].value_counts().index \
    and val in viz_df[viz_df.Label == 1]['BeneficiaryCountry'].value_counts().index:
        palette_map[val] = color
    else:
        palette_map[val]= 'silver'

# Plot countplot of non-anomalous transactions
sns.countplot(x='BeneficiaryCountry', data=viz_df[viz_df.Label == 0], ax=ax1, 
              order=viz_df[viz_df.Label == 0]['BeneficiaryCountry'].value_counts().index,  # Order descending
              palette=palette_map)

# Plot countplot of anomalous transactions
sns.countplot(x='BeneficiaryCountry', data=viz_df[viz_df.Label == 1], ax=ax2, 
              order=viz_df[viz_df.Label == 1]['BeneficiaryCountry'].value_counts().index,  # Order descending
              palette=palette_map)

# Print percentages on top of bars (ax1)
for p in ax1.patches:
    txt = str(round(p.get_height() / viz_df[viz_df.Label == 0].shape[0]*100, 1)) + '%'
    txt_x = p.get_x()+0.12
    txt_y = p.get_height()+350
    ax1.text(txt_x, txt_y, txt, fontsize=12)

# Print percentages on top of bars (ax2)
for p in ax2.patches:
    txt = str(round(p.get_height() / viz_df[viz_df.Label == 1].shape[0]*100, 1)) + '%'
    txt_x = p.get_x()+0.29
    txt_y = p.get_height()+0.7
    ax2.text(txt_x, txt_y, txt, fontsize=12)

ax1.set_title('Beneficiary Entity Countries of Non-Anomalous Transactions (Label 0)', fontsize=17)
ax1.set_xlabel('Beneficiary Entity Country', fontsize=16)
ax1.set_xticklabels(ax1.get_xticklabels(), fontsize=12)
ax1.set_ylabel('Count (5% Sample)', fontsize=16)
ax1.set_yticklabels(['{:,}'.format(int(x)) for x in ax1.get_yticks()], fontsize=12)
ax2.set_title('Beneficiary Entity Countries of Anomalous Transactions (Label 1)', fontsize=17)
ax2.set_xlabel('Beneficiary Entity Country', fontsize=16)
ax2.set_xticklabels(ax2.get_xticklabels(), fontsize=12)
ax2.set_ylabel('Count (5% Sample)', fontsize=16)
ax2.set_yticklabels(ax2.get_yticks().astype(int), fontsize=12);

**Remarks**:
- These visualizations bear resemblance to the sender banks visualization, where we have a large selection of values (countries in this case) among non-anomalous transactions and a much smaller subset of values among anomalous transactions. It does look like there is signal in this feature, given the marked difference between non-anomalous and anomalous transactions for both ordering and beneficiary entity countries.

<br>

## Create `InstructedAmountUSD` feature

Here we are going to create a new column with standardized instructed transaction amounts, as it wouldn't make much sense to leave the amounts in completely different scales. We'll scale all amounts to their USD conversion rates on 2022/01/12, the median transaction date in `train_df`.

Exchange rates were obtained from <a href='https://www.xe.com/currencytables/' >xe.com</a>.

In [16]:
# Exchange rate values on 2022/01/12 (median date of transactions in dataframe)
'1 NZD = 0.6845945591 USD'
'1 GBP = 1.3696247772 USD'
'1 CAD = 0.7999730579 USD'
'1 EUR = 1.1431474656 USD'
'1 AUD = 0.7277363105 USD'
'1 JPY = 0.0087157154 USD'
'1 INR = 0.0135530538 USD'

conversion_rates = {'NZD': 0.6845945591, 'GBP': 1.3696247772, 'CAD': 0.7999730579, 'EUR': 1.1431474656, \
                    'AUD': 0.7277363105, 'JPY': 0.0087157154, 'INR': 0.0135530538, 'USD': 1.0}

# Create new column in train and test dataframes with conversion_rates dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*conversion_rates.items())])

train_df = train_df.withColumn('InstructedAmountUSD', \
                               mapping_expr[F.col('InstructedCurrency')]*F.col('InstructedAmount'))
test_df = test_df.withColumn('InstructedAmountUSD', \
                             mapping_expr[F.col('InstructedCurrency')]*F.col('InstructedAmount'))

# Convert column to integer type
train_df = train_df.withColumn('InstructedAmountUSD', F.round(train_df['InstructedAmountUSD']).cast(IntegerType()))
test_df = test_df.withColumn('InstructedAmountUSD', F.round(test_df['InstructedAmountUSD']).cast(IntegerType()))

In [ ]:
# Display 10 transactions to verify accuracy of InstructedAmountUSD column
train_df.select('InstructedCurrency', 'InstructedAmount', 'InstructedAmountUSD').show(10, truncate=False)

### Visualize target class distributions of instructed transaction amounts in USD

We'll only plot instructed transaction amounts, since settlement amounts are essentially equivalent to instructed amounts less deductions for fees and transfer/forex charges. We'll also only plot the `InstructedAmountUSD` column values, i.e. instructed amounts that have been standardized to the USD exchange rate, as it wouldn't be very informative to plot the wildly varying scales of the different currencies together.

In [ ]:
# Sample 5% of train_df for visualizations (approximately 235k observations) for efficiency
viz_cols = ['InstructedAmountUSD', 'Label']
viz_df = train_df.select(viz_cols).sample(withReplacement=False, fraction=0.05, seed=42).toPandas()

In [ ]:
# Define figure and axes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(22, 9))

# Plot histogram of non-anomalous transactions; log x-axis due to extreme right-skew
sns.histplot(x='InstructedAmountUSD', data=viz_df[viz_df.Label == 0], ax=ax1, bins=10, log_scale=True, color='#85bb65')

# Plot histogram of anomalous transactions; log x-axis due to extreme right-skew
sns.histplot(x='InstructedAmountUSD', data=viz_df[viz_df.Label == 1], ax=ax2, bins=10, log_scale=True, color='#85bb65')

ax1.set_title('Instructed Transaction Amounts of Non-Anomalous Transactions (Label 0)', fontsize=17)
ax1.set_xlabel('Instructed Amount USD (Log-Scale)', fontsize=16)
ax1.set_ylabel('Count (5% Sample)', fontsize=16)
ax2.set_title('Instructed Transaction Amounts of Anomalous Transactions (Label 1)', fontsize=17)
ax2.set_xlabel('Instructed Amount USD (Log-Scale)', fontsize=16)
ax2.set_ylabel('Count (5% Sample)', fontsize=16);

# Set xticks to match bin widths
ax1.set_xticks([bin.get_x() for bin in ax1.patches] + [viz_df[viz_df.Label == 0]['InstructedAmountUSD'].max()])
ax2.set_xticks([bin.get_x() for bin in ax2.patches] + [viz_df[viz_df.Label == 1]['InstructedAmountUSD'].max()]);
# Change xtick labels to more readable format
ax1.set_xticklabels(['$1', '$10', '$200', '$2.8k', '$41k', '$590k', '$8.6M', '$125M', '$1.8B', '$26B', '$380B'], fontsize=12)
ax1.set_yticklabels(['{:,}'.format(int(x)) for x in ax1.get_yticks()], fontsize=12)
ax2.set_xticklabels(['$14k', '$51k', '$180k', '$640k', '$2.3M', '$8.0M', '$28M', '$100M', '$350M', '$1.3B', '$4.4B'], fontsize=12)
ax2.set_yticklabels(ax2.get_yticks().astype(int), fontsize=12);

**Remarks:**
- The distributions of instructed transaction amounts for both non-anomalous and anomalous transactions are both extremely right-skewed, which we can tell because the distributions look more or less normal after log-scaling.
- Non-anomalous transactions appear to have a much larger range of values, ranging from \$1 all the way to over \$380B. The range for anomalous transactions is much more narrow, ranging from \$14k to just under $4.5B.

<br>

## Create `IntermediaryTransactions` feature by counting number of duplicate UETR occurrences for each unique UETR code

This feature will tell us the number of intermediary transactions conducted in each end-to-end transaction. This can be calculated for each end-to-end transaction by counting how many times the UETR for that end-to-end transaction code appears.

In [17]:
# Create temporary table to use for SQL query
train_df.createOrReplaceTempView('train_df_sql')

# Create new feature using SQL
join_sql = """
WITH UETRCounts AS (
SELECT UETR, 
COUNT(UETR)-1 AS IntermediaryTransactions
FROM train_df_sql
GROUP BY UETR
)
SELECT train_df_sql.*, 
UETRCounts.IntermediaryTransactions
FROM train_df_sql
LEFT JOIN UETRCounts
    ON train_df_sql.UETR = UETRCounts.UETR
"""

train_df = spark.sql(join_sql)

### Accuracy check

In [ ]:
# Retrieve random UETR codes of transactions where entity used 0, 1 and 2 intermediary banks
sample1_UETR = train_df.filter(train_df.IntermediaryTransactions == 0).sample(False, 0.00001).limit(1).collect()[0]['UETR']
sample2_UETR = train_df.filter(train_df.IntermediaryTransactions == 1).sample(False, 0.00001).limit(1).collect()[0]['UETR']
sample3_UETR = train_df.filter(train_df.IntermediaryTransactions == 2).sample(False, 0.0001).limit(1).collect()[0]['UETR']

# Display sample of transactions to verify accuracy of OriginalSender and FinalReceiver
#  columns; IntermediaryTransactions value should match number of rows in sample - 1
cols_to_show = ['MessageId', 'UETR', 'Sender', 'Receiver', 'IntermediaryTransactions']
train_df.filter(train_df.UETR == sample1_UETR).select(cols_to_show).show(truncate=False)
train_df.filter(train_df.UETR == sample2_UETR).select(cols_to_show).show(truncate=False)
train_df.filter(train_df.UETR == sample3_UETR).select(cols_to_show).show(truncate=False)

### Repeat for `test_df`

In [18]:
# Create temporary table to use for SQL query
test_df.createOrReplaceTempView('test_df_sql')

# Create new features using SQL
join_sql = """
WITH UETRCounts AS (
SELECT UETR, 
COUNT(UETR)-1 AS IntermediaryTransactions
FROM test_df_sql
GROUP BY UETR
)
SELECT test_df_sql.*, 
UETRCounts.IntermediaryTransactions
FROM test_df_sql
LEFT JOIN UETRCounts
    ON test_df_sql.UETR = UETRCounts.UETR
"""

test_df = spark.sql(join_sql)

<br>

## Create `OriginalSender` and `FinalReceiver` features by identifying original `Sender` and final `Receiver` for each end-to-end transaction

As described in the preliminary EDA section, the correct way to associate ordering and beneficiary accounts with their proper banks is to identify the `Sender` bank in the first transaction of the end-to-end transaction and the `Receiver` bank in the last transaction of the end-to-end transaction. Below, we'll create new columns which do just that.

In [19]:
# Create temporary table to use for SQL query
train_df.createOrReplaceTempView('train_df_sql')

# Create new features using SQL
join_sql = """
WITH EarliestTransaction AS (
SELECT UETR, 
Sender, 
Timestamp
FROM train_df_sql 
WHERE (UETR, Timestamp) IN
    (SELECT UETR, MIN(Timestamp) 
    FROM train_df_sql 
    GROUP BY UETR)
), 
LatestTransaction AS (
SELECT UETR, 
Receiver, 
Timestamp
FROM train_df_sql 
WHERE (UETR, Timestamp) IN
    (SELECT UETR, MAX(Timestamp) 
    FROM train_df_sql 
    GROUP BY UETR)
)
SELECT train_df_sql.*, 
EarliestTransaction.Sender AS OriginalSender, 
LatestTransaction.Receiver AS FinalReceiver
FROM train_df_sql
LEFT JOIN EarliestTransaction
    ON train_df_sql.UETR = EarliestTransaction.UETR
LEFT JOIN LatestTransaction
    ON train_df_sql.UETR = LatestTransaction.UETR
"""

train_df = spark.sql(join_sql)

### Accuracy check

In [ ]:
# Retrieve three random UETR codes of transactions where entity used intermediary banks
sample = train_df.filter(train_df.IntermediaryTransactions > 0).sample(False, 0.0001).limit(3).collect()
UETRs = [sample[row]['UETR'] for row in range(3)]

# Display sample of transactions to verify accuracy of OriginalSender and FinalReceiver columns
cols_to_show = ['Timestamp', 'UETR', 'Sender', 'Receiver', 'OriginalSender', 'FinalReceiver']
train_df.filter(train_df.UETR == UETRs[0]).select(cols_to_show).show(truncate=False)
train_df.filter(train_df.UETR == UETRs[1]).select(cols_to_show).show(truncate=False)
train_df.filter(train_df.UETR == UETRs[2]).select(cols_to_show).show(truncate=False)

### Repeat for `test_df`

In [20]:
# Create temporary table to use for SQL query
test_df.createOrReplaceTempView('test_df_sql')

# Create new features using SQL
join_sql = """
WITH EarliestTransaction AS (
SELECT UETR, 
Sender, 
Timestamp
FROM test_df_sql 
WHERE (UETR, Timestamp) IN
    (SELECT UETR, MIN(Timestamp) 
    FROM test_df_sql 
    GROUP BY UETR)
), 
LatestTransaction AS (
SELECT UETR, 
Receiver, 
Timestamp
FROM test_df_sql 
WHERE (UETR, Timestamp) IN
    (SELECT UETR, MAX(Timestamp) 
    FROM test_df_sql 
    GROUP BY UETR)
)
SELECT test_df_sql.*, 
EarliestTransaction.Sender AS OriginalSender, 
LatestTransaction.Receiver AS FinalReceiver
FROM test_df_sql
LEFT JOIN EarliestTransaction
    ON test_df_sql.UETR = EarliestTransaction.UETR
LEFT JOIN LatestTransaction
    ON test_df_sql.UETR = LatestTransaction.UETR
"""

test_df = spark.sql(join_sql)

### Visualize target class distributions of senders, final senders, receivers, and final receivers

Here we will do a similar visualization as we did in the EDA section, except instead of comparing the same banks across non-anomalous and anomalous transactions, the primary comparison here will be the difference in frequency between how often a given bank is a regular sender vs. original sender, and regular receiver vs. final receiver.

If these visualizations show a great deal of difference between the two across multiple banks, it'll tell us that these features carry a lot of signal and are worth bringing into the final dataframe as one hot encoded features.

In [ ]:
# Sample 5% of train_df for visualizations (approximately 235k observations) for efficiency
viz_cols = ['Sender', 'OriginalSender', 'Label']
viz_df = train_df.select(viz_cols).sample(withReplacement=False, fraction=0.05, seed=42).toPandas()
# Melt dataframe so original/final senders and receivers and can be plotted side-by-side
viz_df = pd.melt(viz_df, id_vars=['Label'], value_vars=['Sender', 'OriginalSender'], var_name='Type', value_name='Bank')

In [ ]:
# Define figure and axes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(23, 14))

# Plot countplot of non-anomalous transactions
sns.countplot(y='Bank', data=viz_df[viz_df.Label == 0], ax=ax1, hue='Type', 
              order=viz_df[viz_df.Label == 0]['Bank'].value_counts().index,  # Order descending
              palette='tab10')

# Plot countplot of anomalous transactions
sns.countplot(y='Bank', data=viz_df[viz_df.Label == 1], ax=ax2, hue='Type', 
              order=viz_df[viz_df.Label == 1]['Bank'].value_counts().index,  # Order descending
              palette='tab10')

# # Print percentages to the right of bars (ax1)
for p in ax1.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width()/viz_df[viz_df.Label == 0].shape[0])
    x = p.get_x() + p.get_width()+100
    y = p.get_y() + p.get_height()-0.15
    ax1.annotate(percentage, (x, y))

# # Print percentages to the right of bars (ax2)
for p in ax2.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width()/viz_df[viz_df.Label == 1].shape[0])
    x = p.get_x() + p.get_width()+0.3
    y = p.get_y() + p.get_height()-0.18
    ax2.annotate(percentage, (x, y))

ax1.set_title('Sender Banks of Non-Anomalous Transactions (Label 0)', fontsize=17)
ax1.set_xlabel('Count (5% Sample)', fontsize=16)
ax1.set_xticklabels(['{:,}'.format(int(x)) for x in ax1.get_xticks()], fontsize=12)
ax1.set_ylabel('Institution (Bank)', fontsize=16)
ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=11)
ax1.legend(title='Type', loc='center right', fontsize=12, title_fontsize=13)
ax2.set_title('Sender Banks of Anomalous Transactions (Label 1)', fontsize=17)
ax2.set_xlabel('Count (5% Sample)', fontsize=16)
ax2.set_xticklabels(ax2.get_xticks().astype(int), fontsize=12)
ax2.set_ylabel('Institution (Bank)', fontsize=16)
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=11)
ax2.legend(title='Type', loc='center right', fontsize=12, title_fontsize=13);

In [ ]:
# Sample 5% of train_df for visualizations (approximately 235k observations) for efficiency
viz_cols = ['Receiver', 'FinalReceiver', 'Label']
viz_df = train_df.select(viz_cols).sample(withReplacement=False, fraction=0.05, seed=42).toPandas()
viz_df = pd.melt(viz_df, id_vars=['Label'], value_vars=['Receiver', 'FinalReceiver'], var_name='Type', value_name='Bank')

In [ ]:
# Define figure and axes
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(23, 14))

# Plot countplot of non-anomalous transactions
sns.countplot(y='Bank', data=viz_df[viz_df.Label == 0], ax=ax1, hue='Type', 
              order=viz_df[viz_df.Label == 0]['Bank'].value_counts().index,  # Order descending
              palette='tab10')

# Plot countplot of anomalous transactions
sns.countplot(y='Bank', data=viz_df[viz_df.Label == 1], ax=ax2, hue='Type', 
              order=viz_df[viz_df.Label == 1]['Bank'].value_counts().index,  # Order descending
              palette='tab10')

# # Print percentages to the right of bars (ax1)
for p in ax1.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width()/viz_df[viz_df.Label == 0].shape[0])
    x = p.get_x() + p.get_width()+100
    y = p.get_y() + p.get_height()-0.15
    ax1.annotate(percentage, (x, y))

# # Print percentages to the right of bars (ax2)
for p in ax2.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width()/viz_df[viz_df.Label == 1].shape[0])
    x = p.get_x() + p.get_width()+0.3
    y = p.get_y() + p.get_height()-0.1
    ax2.annotate(percentage, (x, y))

ax1.set_title('Receiver Banks of Non-Anomalous Transactions (Label 0)', fontsize=17)
ax1.set_xlabel('Count (5% Sample)', fontsize=16)
ax1.set_xticklabels(['{:,}'.format(int(x)) for x in ax1.get_xticks()], fontsize=12)
ax1.set_ylabel('Institution (Bank)', fontsize=16)
ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=11)
ax1.legend(title='Type', loc='center right', fontsize=12, title_fontsize=13)
ax2.set_title('Receiver Banks of Anomalous Transactions (Label 1)', fontsize=17)
ax2.set_xlabel('Count (5% Sample)', fontsize=16)
ax2.set_xticklabels(ax2.get_xticks().astype(int), fontsize=12)
ax2.set_ylabel('Institution (Bank)', fontsize=16)
ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=11)
ax2.legend(title='Type', loc='center right', fontsize=12, title_fontsize=13);

**Remarks**
- While there is definitely a noticeable difference in frequency between original vs. final sender/receiver for some banks, overall the difference is minor for the majority. However, we see an interesting trend where original senders appear to be concentrated among four banks for anomalous transactions, while non-anomalous transactions include many more. This trend reverses though for receiver banks, where there is a greater variety of banks among anomalous transactions than among non-anomalous ones.

<br>

## Create `Flagged` feature by joining `Flags` column of `banks_df`

As we saw in the preliminary EDA section, none of the accounts with non-zero flags are associated with `OrderingAccount`s; they are all associated with `BeneficiaryAccount`s. It therefore wouldn't make any sense to create a column of `OrderingAccontFlag`s as every value in it would be `0`. We'll only make a column of `BeneficiaryAccountFlag`s.

In [21]:
# Create temporary tables to use for SQL query
train_df.createOrReplaceTempView('train_df_sql')
banks_df.createOrReplaceTempView('banks_df_sql')

# Create new feature using SQL
join_sql = """
SELECT train_df_sql.*, 
banks_df_sql.Account AS MatchingBeneficiaryAccount, 
banks_df_sql.Flags AS BeneficiaryAccountFlag
FROM train_df_sql
LEFT JOIN banks_df_sql
    ON train_df_sql.BeneficiaryAccount = banks_df_sql.Account
"""

train_df = spark.sql(join_sql)

### Accuracy check

In [ ]:
# Retrieve three random UETR codes of transactions with non-zero flags
sample = train_df.filter(train_df.BeneficiaryAccountFlag != 0).sample(False, 0.1).limit(3).collect()
UETRs = [sample[row]['UETR'] for row in range(3)]

# Only show relevant columns of train_df
cols_to_show_train = ['Timestamp', 'UETR', 'OrderingAccount', 'BeneficiaryAccount', \
                      'MatchingBeneficiaryAccount', 'BeneficiaryAccountFlag']

# Only show relevant columns of banks_df
cols_to_show_bank = ['Account', 'Flags']

# Get selected beneficiary accounts of train_df to display side-by-side with matching account in banks_df
first_account = train_df.filter(train_df.UETR == UETRs[0]).collect()[0]['BeneficiaryAccount']
second_account = train_df.filter(train_df.UETR == UETRs[1]).collect()[0]['BeneficiaryAccount']
third_account = train_df.filter(train_df.UETR == UETRs[2]).collect()[0]['BeneficiaryAccount']

train_df.filter(train_df.UETR == UETRs[0]).select(cols_to_show_train).show(truncate=False, vertical=True)
banks_df.filter(banks_df.Account == first_account).select(cols_to_show_bank).show(truncate=False, vertical=True)
print('\n')
train_df.filter(train_df.UETR == UETRs[1]).select(cols_to_show_train).show(truncate=False, vertical=True)
banks_df.filter(banks_df.Account == second_account).select(cols_to_show_bank).show(truncate=False, vertical=True)
print('\n')
train_df.filter(train_df.UETR == UETRs[2]).select(cols_to_show_train).show(truncate=False, vertical=True)
banks_df.filter(banks_df.Account == third_account).select(cols_to_show_bank).show(truncate=False, vertical=True)

### Repeat for `test_df`

In [22]:
# Create temporary table to use for SQL query
test_df.createOrReplaceTempView('test_df_sql')

# Create new feature using SQL
join_sql = """
SELECT test_df_sql.*, 
banks_df_sql.Account AS MatchingBeneficiaryAccount, 
banks_df_sql.Flags AS BeneficiaryAccountFlag
FROM test_df_sql
LEFT JOIN banks_df_sql
    ON test_df_sql.BeneficiaryAccount = banks_df_sql.Account
"""

test_df = spark.sql(join_sql)

In [ ]:
# Check for null values in new BeneficiaryAccountFlag column
train_df.where(F.col('BeneficiaryAccountFlag').isNull() | F.isnan('BeneficiaryAccountFlag')).count()

In [23]:
# Replace null values in BeneficiaryAccountFlag with value of '2' which will signify that there is no 
# matching account in banks_df
train_df = train_df.fillna(value=2, subset='BeneficiaryAccountFlag')
test_df = test_df.fillna(value=2, subset='BeneficiaryAccountFlag')

assert train_df.where(F.col('BeneficiaryAccountFlag').isNull() | F.isnan('BeneficiaryAccountFlag')).count() == 0
assert test_df.where(F.col('BeneficiaryAccountFlag').isNull() | F.isnan('BeneficiaryAccountFlag')).count() == 0

In [ ]:
# Check distribution of flagged beneficiary accounts in train_df
train_df.groupBy('Label', 'BeneficiaryAccountFlag').agg({'Label': 'count'}).show()

**Remarks:**
- The math checks out on the number of flagged beneficiary accounts - we saw 800 flagged accounts in the EDA section, and after filling in the 111 missing values with 2s, we have 4,100 zero-flagged anomalous transactions and 800 non-zero-flagged anomalous transactions.
- It looks like there are no non-anomalous beneficiary accounts that are non-zero-flagged. In other words, every single non-anomalous transaction has a flag of 0 associated with the beneficiary account. This doesn't mean the classification will be easy now though; 3,989 out of 4,900 anomalous transactions are also zero-flagged.
- Since every single non-zero-flagged account, regardless of the type of flag, is anomalous, it actually doesn't matter what kind of flag it is as far as predictive modeling goes, so to one hot encode this feature would pointlessly add excess dimensionality to the dataset. Let's replace `BeneficiaryAccountFlag` with a simple binary categorical `Flagged` column: `0` if zero-flagged and `1` if non-zero-flagged.

In [24]:
train_df = train_df.withColumn('Flagged', F.when(train_df.BeneficiaryAccountFlag != 0, 1).otherwise(0))
test_df = test_df.withColumn('Flagged', F.when(test_df.BeneficiaryAccountFlag != 0, 1).otherwise(0))

# Check distribution of new Flagged feature of train_df
train_df.groupBy('Label', 'Flagged').agg({'Label': 'count'}).show()

+-----+-------+------------+
|Label|Flagged|count(Label)|
+-----+-------+------------+
|    1|      0|        3989|
|    1|      1|         911|
|    0|      0|     4686825|
+-----+-------+------------+



<br>

## Create `OrderingCountryFreq` and `BeneficiaryCountryFreq` features

We saw in the creation of `OrderingCountry` and `BeneficiaryCountry` above that there are noticeable difference between non-anomalous and anomalous transactions in the frequency of ordering and beneficiary countries. Let's create numerical features here which capture that difference.

In [25]:
# Create dictionary of ordering country frequency values to map from sender country values
ordering_country_freq = {}
# Create dictionary of beneficiary country currency values to map from beneficiary country values
beneficiary_country_freq = {}

pd_train_df = train_df.select('OrderingCountry', 'BeneficiaryCountry').toPandas()
pd_test_df = test_df.select('OrderingCountry', 'BeneficiaryCountry').toPandas()

for oc in set(
    list(pd_train_df['OrderingCountry'].unique()) + list(pd_test_df['OrderingCountry'].unique())
):
    ordering_country_freq[oc] = len(pd_train_df[pd_train_df['OrderingCountry'] == oc])

for bc in set(
    list(pd_train_df['BeneficiaryCountry'].unique()) + list(pd_test_df['BeneficiaryCountry'].unique())
):
    beneficiary_country_freq[bc] = len(pd_train_df[pd_train_df['BeneficiaryCountry'] == bc])

# Create new column in train and test dataframes with ordering_country_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*ordering_country_freq.items())])

train_df = train_df.withColumn('OrderingCountryFreq', mapping_expr[F.col('OrderingCountry')])
test_df = test_df.withColumn('OrderingCountryFreq', mapping_expr[F.col('OrderingCountry')])

# Create new column in train and test dataframes with beneficiary_country_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*beneficiary_country_freq.items())])

train_df = train_df.withColumn('BeneficiaryCountryFreq', mapping_expr[F.col('BeneficiaryCountry')])
test_df = test_df.withColumn('BeneficiaryCountryFreq', mapping_expr[F.col('BeneficiaryCountry')])

In [ ]:
# Retrieve five random UETR codes of transactions
sample = train_df.sample(False, 0.00001).limit(5).collect()
UETRs = [sample[row]['UETR'] for row in range(5)]

# Display sample of transactions to verify accuracy of new columns
cols_to_show = ['UETR', 'OrderingCountry', 'OrderingCountryFreq', 'BeneficiaryCountry', 'BeneficiaryCountryFreq', 'Label']
train_df.select(cols_to_show).filter(F.col('UETR').isin(UETRs)).show(truncate=15)

<br>

## Create `SenderHourFreq` feature: transaction hour frequency for each `OriginalSender`

This feature will tell us the frequency with which each sender initiated transactions for each hour of the day. This should capture some of the signal of the correlation between the sender and target class as well as the correlation between transaction hour and target class.

For this feature, it was not immediately apparent whether we should use the individual transaction sender (`Sender`) or the end-to-end transaction sender (`OriginalSender`). On the one hand, the transaction sender is the actual bank that is associated with the hour with which that transaction was initiated; on the other hand, the `OriginalSender` is the "correct" bank to associate with the account-holder. In the end, we ended up trying both and doing a correlation analysis, and we found that the `OriginalSender` had a higher correlation with the `Label` column in the training data, so we used the end-to-end `OriginalSender`.

For the testing dataset, we will map the sender hour frequencies from the training dataset; if we were to repeat the process entirely it would result in data leakage.

In [26]:
# Define UDF to extract hour from timestamp
hour = F.udf(lambda x: x.hour, IntegerType())

# Create new column of transaction hours
train_df = train_df.withColumn('Hour', hour(train_df.Timestamp))
test_df = test_df.withColumn('Hour', hour(test_df.Timestamp))

# Create list of unique original senders
senders = train_df.select('OriginalSender').toPandas()['OriginalSender'].unique()

# Create column of senders concatenated with hours
train_df = train_df.withColumn('SenderHour', F.concat(F.col('OriginalSender'), F.col('Hour').cast(StringType())))
test_df = test_df.withColumn('SenderHour', F.concat(F.col('OriginalSender'), F.col('Hour').cast(StringType())))

pd_df = train_df.select('OriginalSender', 'Hour').toPandas()

# Create dictionary of sender hour frequency values to map from sender hour values
sender_hour_frequency = {}

for sender in senders:
    sender_rows = pd_df[pd_df['OriginalSender'] == sender]
    for hour in range(24):
        sender_hour_frequency[sender + str(hour)] = len(sender_rows[sender_rows['Hour'] == hour])

# Create new column in train and test dataframes with sender_hour_frequency dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_hour_frequency.items())])

train_df = train_df.withColumn('SenderHourFreq', mapping_expr[F.col('SenderHour')])
test_df = test_df.withColumn('SenderHourFreq', mapping_expr[F.col('SenderHour')])

In [ ]:
# Retrieve five random UETR codes of transactions
sample = train_df.sample(False, 0.00001).limit(5).collect()
UETRs = [sample[row]['UETR'] for row in range(5)]

# Display sample of transactions to verify accuracy of new columns
cols_to_show = ['UETR', 'OriginalSender', 'Hour', 'SenderHour', 'SenderHourFreq', 'Label']
train_df.select(cols_to_show).filter(F.col('UETR').isin(UETRs)).show(truncate=15)

<br>

## Create `SenderCurrencyFreq` and `SenderCurrencyAmtAvg` features: transaction currency frequency and average transaction amount per currency for each sender

These features will tell us the frequency with which each sender initiated transactions for each currency, in the case of the first feature. For the second feature, it will tell us the average amount with which each sender sent each currency. These features may also be correlated with anomalous transactions.

We'll make sure to use `InstructedAmountUSD` and not `InstructedAmount` to keep a consistent scale. Like before, we will use the `OriginalSender` as we know that the correct way to identify an account with its `Sender` bank is to look at the original sender in the end-to-end transaction.

Additionally, we'll use the training data to map values to the testing data to avoid data leakage (the testing data is converted to a Pandas dataframe and used in the `for` loop only to retrieve `Sender`-currency combinations that might not be in the training data).

In [27]:
# Create column of senders concatenated with instructed currencies
train_df = train_df.withColumn('SenderCurrency', F.concat(F.col('OriginalSender'), F.col('InstructedCurrency')))
test_df = test_df.withColumn('SenderCurrency', F.concat(F.col('OriginalSender'), F.col('InstructedCurrency')))

# Create temporary Pandas dataframes of sender currencies and amounts in USD
pd_train_df = train_df.select('SenderCurrency', 'InstructedAmountUSD').toPandas()
pd_test_df = test_df.select('SenderCurrency', 'InstructedAmountUSD').toPandas()

# Create dictionary of sender currency frequency values to map from sender currency values
sender_currency_freq = {}
# Create dictionary of average sender currency values to map from sender currency values
sender_currency_avg = {}

for sc in set(
    list(pd_train_df['SenderCurrency'].unique()) + list(pd_test_df['SenderCurrency'].unique())
):
    sender_currency_freq[sc] = len(pd_train_df[pd_train_df['SenderCurrency'] == sc])
    sender_currency_avg[sc] = pd_train_df[pd_train_df['SenderCurrency'] == sc][
        'InstructedAmountUSD'
    ].mean()

# Create new column in train and test dataframes with sender_currency_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_currency_freq.items())])

train_df = train_df.withColumn('SenderCurrencyFreq', mapping_expr[F.col('SenderCurrency')])
test_df = test_df.withColumn('SenderCurrencyFreq', mapping_expr[F.col('SenderCurrency')])

# Create new column in train and test dataframes with sender_currency_avg dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_currency_avg.items())])

train_df = train_df.withColumn('SenderCurrencyAmtAvg', mapping_expr[F.col('SenderCurrency')])
test_df = test_df.withColumn('SenderCurrencyAmtAvg', mapping_expr[F.col('SenderCurrency')])

# Convert column to integer type
train_df = train_df.withColumn('SenderCurrencyAmtAvg', F.round(train_df['SenderCurrencyAmtAvg']).cast(IntegerType()))
test_df = test_df.withColumn('SenderCurrencyAmtAvg', F.round(test_df['SenderCurrencyAmtAvg']).cast(IntegerType()))

In [ ]:
# Retrieve five random UETR codes of transactions
sample = train_df.sample(False, 0.00001).limit(5).collect()
UETRs = [sample[row]['UETR'] for row in range(5)]

# Display sample of transactions to verify accuracy of new columns
cols_to_show = ['UETR', 'OriginalSender', 'InstructedCurrency', 'InstructedAmountUSD', 'SenderCurrencyFreq', 'SenderCurrencyAmtAvg', 'Label']
train_df.select(cols_to_show).filter(F.col('UETR').isin(UETRs)).show(truncate=20)

<br>

## Create `SenderFreq` and `ReceiverFreq` features: `OriginalSender` and `FinalReceiver` frequencies for each unique `OriginalSender` and `FinalReceiver`

In [28]:
# Create dictionary of sender bank frequency values to map from original sender banks
sender_freq = {}
# Create dictionary of receiver bank frequency values to map from final receiver banks
receiver_freq = {}

pd_train_df = train_df.select('OriginalSender', 'FinalReceiver').toPandas()
pd_test_df = test_df.select('OriginalSender', 'FinalReceiver').toPandas()

for os in set(
    list(pd_train_df['OriginalSender'].unique()) + list(pd_test_df['OriginalSender'].unique())
):
    sender_freq[os] = len(pd_train_df[pd_train_df['OriginalSender'] == os])

for fr in set(
    list(pd_train_df['FinalReceiver'].unique()) + list(pd_test_df['FinalReceiver'].unique())
):
    receiver_freq[fr] = len(pd_train_df[pd_train_df['FinalReceiver'] == fr])

# Create new column in train and test dataframes with sender_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_freq.items())])

train_df = train_df.withColumn('SenderFreq', mapping_expr[F.col('OriginalSender')])
test_df = test_df.withColumn('SenderFreq', mapping_expr[F.col('OriginalSender')])

# Create new column in train and test dataframes with receiver_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*receiver_freq.items())])

train_df = train_df.withColumn('ReceiverFreq', mapping_expr[F.col('FinalReceiver')])
test_df = test_df.withColumn('ReceiverFreq', mapping_expr[F.col('FinalReceiver')])

In [ ]:
# Retrieve five random UETR codes of transactions
sample = train_df.sample(False, 0.00001).limit(5).collect()
UETRs = [sample[row]['UETR'] for row in range(5)]

# Display sample of transactions to verify accuracy of new columns
cols_to_show = ['UETR', 'OriginalSender', 'SenderFreq', 'FinalReceiver', 'ReceiverFreq', 'Label']
train_df.select(cols_to_show).filter(F.col('UETR').isin(UETRs)).show(truncate=20)

<br>

## Create `SenderReceiverFreq` feature: `OriginalSender`-`FinalReceiver` combination frequency for each sender and receiver pair

This feature will tell us the frequency with which each original sender and final receiver executed transactions to one another. This should capture some of the signal in the `FinalReceiver` column that we haven't captured yet up to this point, since we've mostly been focused on the senders in the transactions.

Once again, we will use the training data to map the values to the testing data. The testing data is only used to retrieve any `Sender`-`Receiver` pairs that might not be in the training data.

In [29]:
# Create column of senders concatenated with receivers
train_df = train_df.withColumn('SenderReceiver', F.concat(F.col('OriginalSender'), F.col('FinalReceiver')))
test_df = test_df.withColumn('SenderReceiver', F.concat(F.col('OriginalSender'), F.col('FinalReceiver')))

# Create dictionary of sender receiver frequency values to map from sender receiver values
sender_receiver_freq = {}

pd_train_df = train_df.select('SenderReceiver').toPandas()
pd_test_df = test_df.select('SenderReceiver').toPandas()

for sr in set(
    list(pd_train_df['SenderReceiver'].unique()) + list(pd_test_df['SenderReceiver'].unique())
):
    sender_receiver_freq[sr] = len(pd_train_df[pd_train_df['SenderReceiver'] == sr])

# Create new column in train and test dataframes with sender_receiver_freq dictionary
mapping_expr = F.create_map([F.lit(x) for x in chain(*sender_receiver_freq.items())])

train_df = train_df.withColumn('SenderReceiverFreq', mapping_expr[F.col('SenderReceiver')])
test_df = test_df.withColumn('SenderReceiverFreq', mapping_expr[F.col('SenderReceiver')])

In [ ]:
# Retrieve five random UETR codes of transactions
sample = train_df.sample(False, 0.00001).limit(5).collect()
UETRs = [sample[row]['UETR'] for row in range(5)]

# Display sample of transactions to verify accuracy of new columns
cols_to_show = ['UETR', 'OriginalSender', 'FinalReceiver', 'SenderReceiver', 'SenderReceiverFreq', 'Label']
train_df.select(cols_to_show).filter(F.col('UETR').isin(UETRs)).show(truncate=20)

<br>

## Drop extraneous columns that will not be used in modeling

We're going to drop all categorical columns here. We'll also drop `InstructedAmount` since it was supplanted with `InstructedAmountUSD`, as well as `SettlementAmount` for the reasons already mentioned.

We'll keep `MessageId` for now as an index to identify unique transactions, but we're going to drop it before modeling.

In [30]:
cols_to_drop = [
    'Timestamp',
    'UETR',
    'Sender',
    'Receiver',
    'TransactionReference',
    'OrderingAccount',
    'OrderingName',
    'OrderingStreet',
    'OrderingCountryCityZip',
    'BeneficiaryAccount',
    'BeneficiaryName',
    'BeneficiaryStreet',
    'BeneficiaryCountryCityZip',
    'SettlementDate',
    'SettlementCurrency',
    'SettlementAmount', 
    'InstructedCurrency', 
    'InstructedAmount', 
    'OrderingCountry', 
    'BeneficiaryCountry', 
    'SenderHour',
    'SenderCurrency',
    'SenderReceiver', 
    'OriginalSender', 
    'FinalReceiver', 
    'MatchingBeneficiaryAccount', 
    'BeneficiaryAccountFlag'
]

train_df = train_df.drop(*cols_to_drop)
test_df = test_df.drop(*cols_to_drop)

## Review dataframes before saving

In [31]:
# Verify we didn't forget to repeat any steps for test_df
assert train_df.columns == test_df.columns

In [32]:
# Display random row of train_df
train_df.sample(False, 0.000001).limit(1).show(truncate=False, vertical=True)

-RECORD 0------------------------------
 MessageId                | TRTGIV4GCF 
 Label                    | 0          
 InstructedAmountUSD      | 27376506   
 IntermediaryTransactions | 2          
 Flagged                  | 0          
 OrderingCountryFreq      | 1002612    
 BeneficiaryCountryFreq   | 684901     
 Hour                     | 10         
 SenderHourFreq           | 79671      
 SenderCurrencyFreq       | 721985     
 SenderCurrencyAmtAvg     | 101235942  
 SenderFreq               | 722100     
 ReceiverFreq             | 678932     
 SenderReceiverFreq       | 372672     



In [33]:
# Display random row of test_df
test_df.sample(False, 0.00001).limit(1).show(truncate=False, vertical=True)

-RECORD 0------------------------------
 MessageId                | TEG9WTNKV4 
 Label                    | 0          
 InstructedAmountUSD      | 8636015    
 IntermediaryTransactions | 0          
 Flagged                  | 0          
 OrderingCountryFreq      | 514627     
 BeneficiaryCountryFreq   | 516561     
 Hour                     | 16         
 SenderHourFreq           | 61227      
 SenderCurrencyFreq       | 514627     
 SenderCurrencyAmtAvg     | 4998351    
 SenderFreq               | 514627     
 ReceiverFreq             | 516570     
 SenderReceiverFreq       | 514627     



In [34]:
# Check for null/NaN values in train_df and test_df
train_df_null = train_df.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c))\
                                 .alias(c) for c in train_df.columns if c != 'Timestamp'])

test_df_null = test_df.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c))\
                               .alias(c) for c in test_df.columns if c != 'Timestamp'])

print('Number of null/NaN values per column, train_df:\n')
train_df_null.show(truncate=False, vertical=True)

print('Number of null/NaN values per column, test_df:\n')
test_df_null.show(truncate=False, vertical=True)

Number of null/NaN values per column, train_df:

-RECORD 0-----------------------
 MessageId                | 0   
 Label                    | 0   
 InstructedAmountUSD      | 0   
 IntermediaryTransactions | 0   
 Flagged                  | 0   
 OrderingCountryFreq      | 0   
 BeneficiaryCountryFreq   | 0   
 Hour                     | 0   
 SenderHourFreq           | 0   
 SenderCurrencyFreq       | 0   
 SenderCurrencyAmtAvg     | 0   
 SenderFreq               | 0   
 ReceiverFreq             | 0   
 SenderReceiverFreq       | 0   

Number of null/NaN values per column, test_df:

-RECORD 0-----------------------
 MessageId                | 0   
 Label                    | 0   
 InstructedAmountUSD      | 0   
 IntermediaryTransactions | 0   
 Flagged                  | 0   
 OrderingCountryFreq      | 0   
 BeneficiaryCountryFreq   | 0   
 Hour                     | 0   
 SenderHourFreq           | 0   
 SenderCurrencyFreq       | 0   
 SenderCurrencyAmtAvg     | 0   
 SenderFreq

**Remarks**:
- No null/NaN values - good to go!

<br>

# Modeling Dataframe 1: `weighted_df`

As we saw above, the training dataset is extremely imbalanced in regards to target class distribution. In order to improve modeling performance, we'll create two different dataframes for modeling: one that sets class weights to use in modeling, which is done through a new column specifying the weights to use in `ClassWeight`, and another that we resample through a combination of over and undersampling.

We'll create the new `ClassWeight` column using the function in helper_functions. The bulk of the code for `create_weight_col` was obtained from <a href="https://towardsdatascience.com/predicting-churn-with-pyspark-ml-d65012e9ab7c" > Estella Zhang </a>.



In [9]:
print(create_weight_col.__doc__)


    Calculates and creates a column of class 
    weights in a PySpark dataframe with an imbalanced 
    binary target class distribution. Assigned 
    class weights are equal to 1 - proportion 
    of class in dataframe.

    Parameters
    ----------
    df : Spark `DataFrame`
        The Spark `DataFrame` to assign the 
        `ClassWeight` column to
    label_col : Spark `Column`
        Label column name
    


In [36]:
train_df = create_weight_col(train_df, label_col='Label')

In [38]:
cols_to_show = ['MessageId', 'Label', 'ClassWeight']
train_df.select(cols_to_show).where(train_df.Label == 0).show(1, truncate=False, vertical=True)
train_df.select(cols_to_show).where(train_df.Label == 1).show(1, truncate=False, vertical=True)

-RECORD 0----------------------------
 MessageId   | TR5B249SIM            
 Label       | 0                     
 ClassWeight | 0.0010443919880214632 
only showing top 1 row

-RECORD 0-------------------------
 MessageId   | TRZPGSV44S         
 Label       | 1                  
 ClassWeight | 0.9989556080119786 
only showing top 1 row



### Save weighted training dataframe and preprocessed test dataframe as CSV files

In [39]:
train_df.coalesce(1).write.csv('/content/drive/MyDrive/Colab Notebooks/train_df_weighted.csv', header=True)
test_df.coalesce(1).write.csv('/content/drive/MyDrive/Colab Notebooks/test_df_preprocessed.csv', header=True)

<br>

# Modeling Dataframe 2: `resampled_df`

For the second dataset, we'll rebalance the data through a combination of undersampling the majority class (non-amomalous transactions) and oversampling the minority class (anomalous transactions). Using the function I wrote in helper_functions.py, we'll specify a more balanced positive to negative class ratio of 0.25 in the new resampled dataframe, with 4,690,000 observations (roughly equivalent to the number of rows in the original training dataset).

In [10]:
print(spark_resample.__doc__)


    Resamples PySpark dataframe with an imbalanced binary 
    target class distribution through a combination of 
    undersampling the majority (negative) class and 
    oversampling the minority (positive) class.

    Parameters
    ----------
    df : Spark `DataFrame`
        The Spark `DataFrame` to resample
    ratio : `float`
        Desired ratio of positive (oversampled) class to 
        negative (undersampled) class in new `DataFrame`
    new_count : `int`
        Desired total count of observations (rows) in new
        `DataFrame`
    class_field : Spark `Column`
        Name of `Column` in `DataFrame` to resample
    pos_class : `DataType` of `class_field`
        Label of positive class in `class_field`
    shuffle : `bool`
        Whether to shuffle `DataFrame` before returning 
        resampled `DataFrame`. If `False`, positive 
        class observations will all be distributed on 
        the bottom of the resampled `DataFrame`.
    random_state : `int`
        Ra

In [43]:
# Drop ClassWeight column of train_df
train_df = train_df.drop('ClassWeight')

# Resample train_df; specify approximately 4,690,000 observations in new resampled dataframe, 
# with more balanced (0.25) ratio of positive target class (1) to negative target class (0)
train_df = spark_resample(train_df, ratio=0.25, new_count=4690000, 
                          class_field='Label', pos_class=1, shuffle=True, random_state=42)

In [45]:
# Print shape of resampled dataframe
print(f"Resampled train_df:  {train_df.count():,} Rows, {len(train_df.columns)} Columns")

Resampled train_df:  4,690,821 Rows, 14 Columns


In [47]:
# Preview resampled dataframe
train_df.show(3, truncate=False, vertical=True)

-RECORD 0------------------------------
 MessageId                | TRU6CKHUXQ 
 Label                    | 0          
 InstructedAmountUSD      | 809115     
 IntermediaryTransactions | 0          
 Flagged                  | 0          
 OrderingCountryFreq      | 49092      
 BeneficiaryCountryFreq   | 116098     
 Hour                     | 5          
 SenderHourFreq           | 11206      
 SenderCurrencyFreq       | 49092      
 SenderCurrencyAmtAvg     | 1907059    
 SenderFreq               | 49092      
 ReceiverFreq             | 116135     
 SenderReceiverFreq       | 49092      
-RECORD 1------------------------------
 MessageId                | TRII65E34T 
 Label                    | 0          
 InstructedAmountUSD      | 24221767   
 IntermediaryTransactions | 0          
 Flagged                  | 0          
 OrderingCountryFreq      | 603157     
 BeneficiaryCountryFreq   | 56642      
 Hour                     | 6          
 SenderHourFreq           | 114677     


In [48]:
# Display value counts for 'Label' column (classification target) of resampled dataframe
resampled_class_counts = train_df.groupBy('Label')\
                                 .count()\
                                 .withColumn('percent', F.col('count')/train_df.count())

resampled_class_counts.show(truncate=10)

+-----+-------+----------+
|Label|  count|   percent|
+-----+-------+----------+
|    0|3517454|0.74985...|
|    1|1173406|0.25014...|
+-----+-------+----------+



### Save resampled training dataframe as CSV file

In [49]:
train_df.coalesce(1).write.csv('/content/drive/MyDrive/Colab Notebooks/train_df_resampled.csv', header=True)